# Bibliotecas Usadas

In [90]:
import numpy as np
import random
import copy

# O Jogo

Abaixo, está a classe elaborada para simular o jogo slide number, necessário para fazer as possibilidades de partida 

In [98]:
class Jogo:
    
    """
    Para o construtor, é necessário informar a dimensão, do tabuleiro quadrado, e uma jogada anterior
    que, por default, não existe. O construtor faz um tabuleiro gabarito
    """
    def __init__(self, dimensao, jogada_anterior=None):
        
        tabuleiro = []
        k=1
        
        #loop tradicional para fazer uma matriz
        for i in range(0,dimensao):
            tabuleiro.append([])
            for j in range(0, dimensao):
                tabuleiro[i].append(k)
                k+=1
        
        #Espaço vazio do jogo                        
        tabuleiro[-1][-1] = None
        
        #posição do vazio
        self.vazio = (dimensao-1, dimensao-1)
        
        #tabuleiro
        self.tabuleiro = tabuleiro
        
        #dimensão do tabuleiro
        self.dimensao = dimensao
        
        #jogada anterior que gerou o tabuleiro atual
        self.jogada_anterior = jogada_anterior
    
    """
    Função para copiar um tabuleiro e mudar sua referencia
    """
    def copy(self):
        return copy.deepcopy(self)
    
    """
    Função que simula uma jogada legal. Retorna outro tabuleiro com a jogada
    """
    def jogada(self, pos):
        
        #presenvando o tabuleiro atual
        novo = self.copy()
        
        #pegando os vizinhos do vazio
        vizinhos = novo.getVizinhos()
        
        #se a jogada for um vizinho, a pos é valida!
        if pos in vizinhos:
            
            novo.tabuleiro[self.vazio[0]][self.vazio[1]] = novo.tabuleiro[pos[0]][pos[1]]
            
            novo.tabuleiro[pos[0]][pos[1]] = None
            
            novo.vazio = pos
            
            novo.jogada_anterior = pos
            
        return novo
    
    """
    Função que retorna as posições de jogadas valídas
    """
    def getVizinhos(self):
        
        #vendo os vizinhos do vazio
        pos = self.vazio
        possibilidades = [
            (pos[0]-1, pos[1]),
            (pos[0]+1, pos[1]),
            (pos[0], pos[1]+1),
            (pos[0], pos[1]-1)
        ]
        
        #tirando os que são ilegais
        possibilidades = filter(lambda x: (x[0] >= 0 and x[0] < self.dimensao) and 
               (x[1] >= 0 and x[1] < self.dimensao), possibilidades)
        
        return list(possibilidades)
    
    """
    Função que cria um tabuleiro randomico partindo de um gabarito
    """    
    def embaralha(self, vezes):
        
        #presenvando o tabuleiro atual
        proximo = self.copy()
        
        #fazundo multiplas jogadas aleatorias
        for i in range(0,vezes):
            x = proximo.getVizinhos()
            
            pos = random.choice(x)
            
            proximo = proximo.jogada(pos)
            
        
        return proximo
    
    """
    Função para verificar igualdade entre tabuleiros
    """
    def __eq__(self, outro):
        return (self.vazio == outro.vazio and self.tabuleiro == outro.tabuleiro)
    
    """
    Sobreescrita do print usando numpy
    """
    def __str__(self):
        return str(np.array(self.tabuleiro))
    

# O nó da árvore

Para montar a arvore, é necessário utilizar uma estrutura de nós com n filhos e 1 pai

In [99]:
class No:
    
    """
    Para construir o nó, é necessário passar seu conteúdo e seu pai que, por default, é None
    """
    def __init__(self, conteudo,pai = None):
        
        #pai do nó
        self.pai = pai
        
        #conteudo do nó
        self.conteudo = conteudo
        
        #lista de filhos
        self.filhos = list()
    
    """
    Funçaõ para adicionar filhos ao nó, deve-se passar apenas o conteudo do filho
    """
    def add(self, conteudo):    
        filho = No(conteudo, pai = self)
        self.filhos.append(filho)
        
    """
    dois nós são iguais se se o conteúdo for o mesmo
    """
    def __eq__(self, outro):
        
        return (self.conteudo == outro.conteudo)

# Funções auxiliares

Para fazer o busca em largura de forma dinamica, foi necessário elaborar funções de verificação e manuntenção de arvore

In [108]:
"""
Função para verificar se, dado um nó, ele ja apareceu em sua linhagem
"""
def Correspondencia(no):
    if not no.pai:
        return False
    
    proximo = no.pai
    
    while proximo:
        
        if proximo == no:
            return True
        else:
            proximo = proximo.pai
    return False

In [109]:
"""
Função para, dado um nó, gerar seus filhos de jogadas lícitas
"""
def CriarFilhos(no):
    t = no.conteudo
    
    pos = t.getVizinhos()
    
    filhos = []

    for coord in pos:
        
        j = t.jogada(coord)
        
        if not Correspondencia(No(j,pai = no)): filhos.append(No(j,pai = no))
    
    return filhos

# A Busca Em Largura Dinâmica

Finalmente, o algoritmo de busca. Ele gera uma arvore de possibilidades até achar uma resposta

In [110]:
"""
Algoritmo para achar o caminho de jogadas do jogo passado como parametro
"""
def BuscaEmLargura(problema, solucao):
    
    #se o seu problema é igual a solução...
    if problema == solucao:
        return problema
    
    #Gerarndo as primeiras possibilidades
    proximos = CriarFilhos(problema)
    
    #enquanto tiver possibilidades
    while len(proximos) != 0:
        
        filho = proximos.pop(0)
        
        #achei a solucao!
        if filho == solucao:
            return filho
        
        for x in CriarFilhos(filho): proximos.append(x)
    
    return None        

In [170]:
"""
Algoritmo para achar o conjunto de jogadas que originaram o nó
"""
def coletarJogadas(no):
    jogadas = []
    proximo = no
    while proximo.pai:
        jogadas.append(proximo.conteudo.jogada_anterior)
        proximo = proximo.pai
    return jogadas[::-1]        

# Demonstração

In [152]:
gabarito = Jogo(3)
print(gabarito)

[[1 2 3]
 [4 5 6]
 [7 8 None]]


In [153]:
problema = gabarito.embaralha(1000)
print(problema)

[[7 1 5]
 [6 8 4]
 [None 3 2]]


In [154]:
inicio = No(problema)
fim = No(gabarito)

In [161]:
resultado = BuscaEmLargura(inicio, fim) 

In [162]:
print(resultado.conteudo)

[[1 2 3]
 [4 5 6]
 [7 8 None]]


In [171]:
jogadas = coletarJogadas(resultado)
print(jogadas)

[(1, 0), (0, 0), (0, 1), (1, 1), (1, 2), (2, 2), (2, 1), (2, 0), (1, 0), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2)]
